In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix,mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import os
import math

In [3]:
path = "../data"

In [159]:
df = pd.read_csv(os.path.join(path,"spotifyqueriesresults.csv"),index_col = None).drop("Unnamed: 0",axis =1)
#prune bad lists
df = df[df['playlist_id'] != "44ihc7hk5ewgfWsy5x9QKL"] 
df = df[df['playlist_id'] != "5LjyNDmQhoODlJZaNV7fxj"]
df = df[df["playlist_id"] != "6s9wratabPxp4NwDZAbnwl"]
df = df[df["playlist_id"] != "69CBhCYkS3jzhmzRoghVWG"]
df = df[df["playlist_id"] != "4MRjoBGxVSA8JcBiJr6C3Y"]
df = df[df["playlist_id"] != "171HLt90V1fhorEJWF45VO"]
df = df[df["playlist_id"] != "4XHLVq8ke4mxKeZvqgJjNZ"]
df = df[df["playlist_id"] != "2fFSLnFhz5nem6mWRWEJaD"]

In [160]:
df.groupby("category").mean()

,acousticness,danceability,energy,instrumentalness,key,loudness,mode,speechiness,tempo,valence
category,,,,,,,,,,
joyful activation,0.168452,0.673130,0.711562,0.090740,5.498336,-6.984493,0.537438,0.112456,119.615384,0.536217
nostalgia,0.479898,0.555012,0.449560,0.217544,5.205546,-10.844982,0.693312,0.064300,115.925433,0.372304
peacefullness,0.679829,0.484904,0.284925,0.493209,5.140549,-16.803195,0.665590,0.065582,109.501991,0.279912
power,0.217689,0.623068,0.668679,0.104055,5.213365,-7.638308,0.630715,0.102970,121.651104,0.493260
sadness,0.404430,0.589158,0.507265,0.029341,5.296602,-8.320311,0.684114,0.086441,118.146968,0.348660
tenderness,0.408639,0.593442,0.480337,0.109199,5.104010,-8.967726,0.547619,0.087788,113.502149,0.368376
tension,0.183656,0.599337,0.708396,0.056457,5.171239,-6.524758,0.663198,0.108359,123.853668,0.484920
transcendence,0.403627,0.506168,0.498595,0.233230,5.141631,-11.564384,0.644635,0.059453,115.233833,0.448944
wonder,0.220666,0.651592,0.660450,0.060559,5.260664,-6.736567,0.676935,0.083493,119.018748,0.554548


In [161]:
df.shape

(9894, 15)

## Baseline randomforest

In [162]:
model_frame = df.drop(["artist","class","playlist_id","title"],axis = 1)

In [163]:
X = model_frame.drop("category",axis =1)
y = model_frame["category"]

In [164]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [165]:
clf = RandomForestClassifier()

In [166]:
clf.fit(X_train,y_train)

RandomForestClassifier()

In [167]:
y_pred = clf.predict(X_test)

In [168]:
print(classification_report(y_test,y_pred))

                   precision    recall  f1-score   support

joyful activation       0.41      0.43      0.42       255
        nostalgia       0.29      0.27      0.28       256
    peacefullness       0.56      0.52      0.54       260
            power       0.19      0.12      0.14       173
          sadness       0.28      0.36      0.31       196
       tenderness       0.23      0.20      0.21       147
          tension       0.31      0.28      0.29       202
    transcendence       0.42      0.50      0.46       231
           wonder       0.34      0.38      0.36       259

         accuracy                           0.36      1979
        macro avg       0.34      0.34      0.34      1979
     weighted avg       0.35      0.36      0.35      1979



## Baseline XGBoost

In [169]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)

In [170]:
print(classification_report(y_test,y_pred))

                   precision    recall  f1-score   support

joyful activation       0.40      0.48      0.44       255
        nostalgia       0.29      0.25      0.27       256
    peacefullness       0.64      0.50      0.56       260
            power       0.23      0.08      0.12       173
          sadness       0.24      0.42      0.30       196
       tenderness       0.30      0.15      0.20       147
          tension       0.30      0.21      0.25       202
    transcendence       0.42      0.52      0.46       231
           wonder       0.32      0.40      0.35       259

         accuracy                           0.35      1979
        macro avg       0.35      0.34      0.33      1979
     weighted avg       0.36      0.35      0.34      1979



## TF-IDF
$w$ = $tf$ $\times$ $log{\frac{N}{docfreq}}$

"TF-IDF compares the number of times a word appears in a doc with the number of docs the word appears in"

in our case,words are musicians and docss are playlist categories

N = total playlists with label $j$

tf = number of times artists $i$ is in playlist category $j$

docfreq = number of playlists that have artist $i$

In [51]:
def tf_idf(artist,label):
    tf = len(df[df.category == label ][df.artist == artist])
    N = len(df[df.category == label ])
    docfreq = len(df[df.artist == "Drake" ])
    
    return tf * math.log(N/docfreq)

In [176]:
#example
tf_idf("Lil Uzi Vert","wonder")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


38.114130925289174

In [174]:
tf_idf("Lil Uzi Vert","sadness")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


18.304053063788782

In [198]:
df[df.category == "nostalgia"].sample(10)

,acousticness,artist,class,danceability,energy,instrumentalness,key,loudness,mode,playlist_id,speechiness,tempo,title,valence,category
3777,0.011000,Timmy Thomas,mellow,0.520,0.669,0.000048,0,-12.925,1,2fkvYeHn5q2Um0pjROecni,0.0417,203.097,(Dying Inside) To Hold You - Heartbreak Radio ...,0.694,nostalgia
3613,0.167000,Radiohead,mellow,0.454,0.229,0.102000,2,-12.592,1,37i9dQZF1DXb9LIXaj5WhL,0.0297,73.552,Fake Plastic Trees,0.134,nostalgia
4335,0.712000,Julie Anne San Jose,soppy,0.315,0.458,0.000003,6,-6.551,1,37i9dQZF1DWX79NXMEFYhh,0.0313,134.157,You Are My Everything - From 'Descendants of t...,0.237,nostalgia
3617,0.006190,Candlebox,mellow,0.591,0.638,0.000005,7,-6.958,1,37i9dQZF1DXb9LIXaj5WhL,0.0290,131.113,Cover Me,0.348,nostalgia
3907,0.740000,Rufus Longspring,dreamy,0.425,0.271,0.952000,2,-20.170,0,0IGSLI6eeoLaVJlqhOHbfm,0.0283,139.952,Morning Boogy,0.312,nostalgia
4160,0.086900,The Velvet Underground,dreamy,0.506,0.442,0.002390,5,-13.268,0,3Cadj8B9xWjygHqZfLDJMc,0.0276,105.826,Sunday Morning,0.532,nostalgia
3705,0.229000,Max RM,mellow,0.623,0.456,0.097700,2,-9.027,0,2uEUJvzzfMKHBV0Y1Cseh3,0.0320,140.064,I Can See Your Smile,0.484,nostalgia
4188,0.442000,Lou Reed,dreamy,0.458,0.502,0.000006,5,-10.186,1,3Cadj8B9xWjygHqZfLDJMc,0.0294,121.861,Satellite of Love,0.571,nostalgia
3763,0.000204,Firehouse,mellow,0.472,0.480,0.002750,1,-11.893,1,2fkvYeHn5q2Um0pjROecni,0.0308,123.023,Love of a Lifetime,0.126,nostalgia
3360,0.826000,Louis Armstrong,mellow,0.568,0.122,0.001360,0,-10.547,1,37i9dQZF1DWTQwRw56TKNc,0.0702,70.775,La Vie En Rose - Single Version,0.325,nostalgia


In [184]:
df[df.playlist_id == "6GUbWs9UAjQVJSS3FkZZNY"]

,acousticness,artist,class,danceability,energy,instrumentalness,key,loudness,mode,playlist_id,speechiness,tempo,title,valence,category
6872,0.052700,Jacquees,get down,0.540,0.555,0.000000,4,-9.858,0,6GUbWs9UAjQVJSS3FkZZNY,0.0720,127.154,"Feel It (feat. Lloyd, Rich Homie Quan)",0.1550,joyful activation
6873,0.294000,Rihanna,get down,0.580,0.532,0.000000,1,-5.668,0,6GUbWs9UAjQVJSS3FkZZNY,0.0424,112.049,Sex With Me,0.3900,joyful activation
6874,0.024200,Lil Peep,get down,0.642,0.538,0.000000,8,-7.206,1,6GUbWs9UAjQVJSS3FkZZNY,0.0696,143.921,witchblades,0.5700,joyful activation
6875,0.221000,Comethazine,get down,0.831,0.795,0.000000,1,-6.186,1,6GUbWs9UAjQVJSS3FkZZNY,0.2640,160.063,Walk,0.7440,joyful activation
6876,0.086500,Various Artists,get down,0.674,0.888,0.000000,1,-5.249,1,6GUbWs9UAjQVJSS3FkZZNY,0.3560,79.772,Talent Show,0.6860,joyful activation
6877,0.008820,Sage The Gemini,get down,0.831,0.504,0.000000,4,-5.846,0,6GUbWs9UAjQVJSS3FkZZNY,0.0532,92.540,Buss It (feat. Chris Brown),0.2600,joyful activation
6878,0.151000,Ty Dolla $ign,get down,0.707,0.543,0.000000,8,-4.610,1,6GUbWs9UAjQVJSS3FkZZNY,0.2790,144.180,Purple Emoji (feat. J. Cole),0.8240,joyful activation
6879,0.200000,Sueco,get down,0.753,0.538,0.000029,10,-10.704,1,6GUbWs9UAjQVJSS3FkZZNY,0.3100,84.034,dork,0.6070,joyful activation
6880,0.432000,Kamaiyah,get down,0.634,0.669,0.000000,0,-7.369,1,6GUbWs9UAjQVJSS3FkZZNY,0.0682,98.044,Fuck It up (feat. Yg),0.4310,joyful activation
6881,0.074100,Bhad Bhabie,get down,0.908,0.625,0.000000,1,-4.519,1,6GUbWs9UAjQVJSS3FkZZNY,0.1900,101.993,Bestie (feat. Kodak Black),0.6130,joyful activation
